# Content-based (Usando modelo BERT y sentence transformers)


En este proyecto trabajaremos con un modelo de recomendacion de libros de la página [Goodreads](http://www.goodreads.com). El modelo de recomendación de libros es un recomendador basado en contenido, donde se utilizan modelos de lenguage BERT y sentence transformers para el cálculo de embeddings de los libros y luego similaridades de ítems. Luego, dependiendo de los libros con los que el usuario ha interactuado, se recomiendan los ítems más similares.

In [ ]:
import numpy as np
import random
import json
import requests
import heapq
import math
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from io import BytesIO
import pickle
import pandas as pd
import time

In [ ]:
!wget --no-cache --backups=1 "https://raw.githubusercontent.com/valegrajales/laboratorio4-sr-uniandes/main/data/book_data.parquet.gzip" -O book_data.parquet.gzip
!wget --no-cache --backups=1 "https://raw.githubusercontent.com/valegrajales/laboratorio4-sr-uniandes/main/data/goodreads_interactions.csv" -O goodreads_interactions.csv
!wget --no-cache --backups=1 "https://raw.githubusercontent.com/valegrajales/laboratorio4-sr-uniandes/main/data/book_descripcion_vectors_bert" -O book_descripcion_vectors_bert
!wget --no-cache --backups=1 "https://raw.githubusercontent.com/valegrajales/laboratorio4-sr-uniandes/main/data/book_descripcion_vectors_mpnet" -O book_descripcion_vectors_mpnet
!wget --no-cache --backups=1 "https://raw.githubusercontent.com/valegrajales/laboratorio4-sr-uniandes/main/data/book_descripcion_vectors_minilm" -O book_descripcion_vectors_minilm

# Cargar datos de libros

In [ ]:
df_books = pd.read_parquet("book_data.parquet.gzip")
df_books.head()

## **Pregunta 1**

Realice el análisis exploratorio de datos, aplicando las mismas técnicas vistas en el laboratorio 2

In [ ]:
# Seleccionar columnas, id de libro, ttulo y descripción, con la columna descripción se generan los vectores (embedding) con información semántica
df_books = df_books[['book_id','title', 'description']]

In [ ]:
df_books.shape

# Carga de interacciones de usuarios con libros

In [ ]:
df_books_interactions = pd.read_csv('goodreads_interactions.csv')
df_books_interactions.head()

In [ ]:
df_books_interactions.shape

In [ ]:
# Separar el conjunto que queremos usar para evaluar el desempeño de los modelos
df_books_interactions_train, df_books_interactions_test = train_test_split(df_books_interactions, test_size=0.2, random_state=42)

# Carga de vectores (embedding): BERT-large y MPNet

En esta sección se trabajará con modelos pre-entrenados de modelos de lenguage BERT-large y MPNet que convierten texto a embeddings. 

Bidirectional Encoder Representations from Transformers (BERT) es una técnica de NLP (Natural Language Processing) desarrollada por Google y publicada en 2018 por Jacob Devlin. 

Actualmente Google utiliza BERT para entender las consultas de los usuarios en su buscador. 

Tiene dos versiones: 
- **BERT:** 12 capas, 12 cabezales de atencion y 110 millones de parámetros. Genera vectores de 768 dimensiones 
- **BERT-large:** 24 capas, 16 cabezales de atencion y 340 millones de parámetros.  

![BERT y BERT-large](http://jalammar.github.io/images/bert-base-bert-large.png)

![BERT y BERT-large arquitectura](http://jalammar.github.io/images/bert-base-bert-large-encoders.png)

MPNet: Masked and Permuted Pre-training for Language Understanding, de Kaitao Song, Xu Tan, Tao Qin, Jianfeng Lu, Tie-Yan Liu, es un novedoso método de preentrenamiento para tareas de comprensión del lenguaje. Resuelve los problemas de MLM (modelado del lenguaje enmascarado) en BERT y PLM (modelado del lenguaje permutado) en XLNet y consigue una mayor precisión. [MPNet](https://arxiv.org/pdf/1905.02450.pdf)

En este caso los textos que utilizaremos son las descripciones de los libros y compararemos los resultados de recomendación con BERT-large y MPNet. El primer paso es generar los vectores de características (embedding) para luego esta representación dimensional semántica en la búsqueda de contenido.

Para mayores detalles sobre el modelo de lenguaje BERT se recomienda revisar el siguiente artículo:
- [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)

# Carga de vectores usando modelo Bert

In [ ]:
# Reducir cantidad de datos a vectorizar, vamos a usar una muestra de 2000 libros, no modifique la semilla, ya que la muestra de interacciones a evaluar depende de la misma y los vectores se generaron con base en esta semilla
df_books_small = df_books.sample(n=10000, random_state=42).fillna('')
df_books_small.shape

In [ ]:
# Diccionario indice a libro id y viceversa para hacer las recomendaciones las recomendaciones 
idx2bookid = {i: id_ for i, id_ in enumerate(df_books_small.book_id)}
bookid2idx = {id_:i for i, id_ in enumerate(df_books_small.book_id)}

In [ ]:
# Cargar los vectores del modelo Bert
with open('book_descripcion_vectors_bert', "rb") as fIn:
  cache_data = pickle.load(fIn)
  book_descripcion_vectors_bert = cache_data['embeddings']

In [ ]:
# Cargar los vectores del modelo MPNet
with open('book_descripcion_vectors_mpnet', "rb") as fIn:
  cache_data = pickle.load(fIn)
  book_descripcion_vectors_mpnet = cache_data['embeddings']

In [ ]:
# Cargar los vectores del modelo MiniLM
with open('book_descripcion_vectors_minilm', "rb") as fIn:
  cache_data = pickle.load(fIn)
  book_descripcion_vectors_minilm = cache_data['embeddings']

## **Pregunta 2** 

Considerando que haremos un recomendador basado en contenidos ¿Por qué el uso de modelos de lenguage es una buena elección para este tipo de problema?

# Probamos con BERT, MPNet y MiniLM reduciendo dimensionalidad con PCA-20

Una vez calculados (o cargados) los vectores característicos de cada libro a partir de su descripción, reducimos dimensionalidad. Probaremos con BERT, MPNet y MiniLM para comparar los resultados en recomendación basada en contenido. 

In [ ]:
# Project into a 20 PCA feature space
pca20_bert_featvectors = PCA(n_components=20).fit_transform(book_descripcion_vectors_bert)
pca20_mpnet_featvectors = PCA(n_components=20).fit_transform(book_descripcion_vectors_mpnet)
pca20_minilm_featvectors = PCA(n_components=20).fit_transform(book_descripcion_vectors_minilm)

In [ ]:
pca20_bert_featvectors.shape

In [ ]:
pca20_mpnet_featvectors.shape

In [ ]:
pca20_minilm_featvectors.shape

### **Pregunta 3**

Comente por qué se utiliza PCA (investigue) para reducir la dimensión de cada vector característico. ¿Qué sucede con la pérdida de información en la reducción de dimensionalidad?

# Recuperación de documentos similares 

En esta sección utilizaremos los vectores cargados para hacer un sistema de recuperación o búsqueda de información, para diferentes métricas de distancia.

Buscamos libros similares de acuerdo a la representación vectorial (BERT) de su descripción.

In [ ]:
# formato de resultados 
pd.options.display.max_colwidth = 50
pd.set_option('display.max_colwidth', None)

In [ ]:
# Find similar books by book id
def find_similar_books(embedding, query_id=None, metric='euclidean', topk=10):
    
    n = embedding.shape[0]
    
    if query_id is None:
        query_i = np.random.randint(n)
        query_id = idx2bookid[query_i]
    
    else:
        query_i = bookid2idx[query_id]
        
    
    distances = pairwise_distances(embedding[query_i].reshape(1,-1), embedding, metric=metric)
    heap = []
    for i in range(n):            
        if len(heap) < topk:
            heapq.heappush(heap, (-distances[0][i], i))
        else:
            heapq.heappushpop(heap, (-distances[0][i], i))

    heap.sort(reverse=True)
    rec_ids = [idx2bookid[i] for _,i in heap]
    
    return rec_ids

## Usando BERT

In [ ]:
# libros similares al libro de id 27421523 (Harry Potter and the Sorcerer's Stone) utilizando distancia euclideana. se puede cambiar a "cosine" 
similar_books = find_similar_books(book_descripcion_vectors_bert, query_id = '27421523', metric = 'euclidean', topk=10 )
similar_books

In [ ]:
df_books_small[df_books_small.book_id.isin(similar_books)][['book_id', 'title', 'description']]

## Usando BERT reducidos con PCA 

In [ ]:
# libros similares al libro de id 27421523 (Harry Potter and the Sorcerer's Stone) utilizando distancia euclideana. se puede cambiar a "cosine" 
similar_books = find_similar_books(pca20_bert_featvectors, query_id = '27421523', metric = 'euclidean', topk=10 )
similar_books

In [ ]:
df_books_small[df_books_small.book_id.isin(similar_books)][['book_id', 'title', 'description']]

## Usando MPNet

In [ ]:
# libros similares al libro de id 27421523 (Harry Potter and the Sorcerer's Stone) utilizando distancia euclideana. se puede cambiar a "cosine" 
similar_books = find_similar_books(book_descripcion_vectors_mpnet, query_id = '27421523', metric = 'euclidean', topk=10 )
similar_books

In [ ]:
df_books_small[df_books_small.book_id.isin(similar_books)][['book_id', 'title', 'description']]

## Usando MPNet reducido con PCA

In [ ]:
# libros similares al libro de id 27421523 (Harry Potter and the Sorcerer's Stone) utilizando distancia euclideana. se puede cambiar a "cosine" 
similar_books = find_similar_books(pca20_mpnet_featvectors, query_id = '27421523', metric = 'euclidean', topk=10 )
similar_books

In [ ]:
df_books_small[df_books_small.book_id.isin(similar_books)][['book_id', 'title', 'description']]

## Usando MiniLM

In [ ]:
# libros similares al libro de id 27421523 (Harry Potter and the Sorcerer's Stone) utilizando distancia euclideana. se puede cambiar a "cosine" 
similar_books = find_similar_books(book_descripcion_vectors_minilm, query_id = '27421523', metric = 'euclidean', topk=10 )
similar_books

In [ ]:
df_books_small[df_books_small.book_id.isin(similar_books)][['book_id', 'title', 'description']]

## Usando MiniLM reducido con PCA

In [ ]:
# libros similares al libro de id 27421523 (Harry Potter and the Sorcerer's Stone) utilizando distancia euclideana. se puede cambiar a "cosine" 
similar_books = find_similar_books(pca20_minilm_featvectors, query_id = '27421523', metric = 'euclidean', topk=10 )
similar_books

In [ ]:
df_books_small[df_books_small.book_id.isin(similar_books)][['book_id', 'title', 'description']]

## Pregunta 4: 
Comente los resultados obtenidos, en cuanto a modelo de lenguaje, reduccion de dimensionalidad y métrica de distancia utilizada.

# Recomendaciones 

In [ ]:
# formato de resultados 
pd.options.display.max_colwidth = 50
pd.set_option('display.max_colwidth', None)

In [ ]:
def recommend(embedding, user_id=None, topk=10, metric='cosine'):
    
    #print("user_id = ", user_id)
    
    #user_id = str(user_id)
    
    #Calculate distance metrics
    trx = df_books_interactions.loc[df_books_interactions['user_id'] == user_id].book_id.tolist()
    #trx = df_books_interactions[user_id]
    n = embedding.shape[0]
    distances = 1e9
    
    # recorremos transacciones pasadas del usuario 
    for t in trx:
        query_i = bookid2idx[str(t)]
        
        # recomendamos items más cercanos a items con los que interactuó el usuario
        distances = np.minimum(distances, pairwise_distances(
                embedding[query_i].reshape(1,-1), embedding, metric=metric).reshape(-1))

    #Rank items de menor a mayor distancia (nos quedamos con los topk)
    trx_set = set(trx)
    heap = []
    for i in range(n):
        if idx2bookid[i] in trx_set:
            continue
        if len(heap) < topk:
            heapq.heappush(heap, (-distances[i], i))
        else:
            heapq.heappushpop(heap, (-distances[i], i))
    heap.sort(reverse=True)
    
    # utilizamos un heap para extraer los items ordenados de menor a mayor distancia 
    recommended_ids = [idx2bookid[i] for _,i in heap]
    
    # retornar los que el usuario no haya consumido 
    filtered_recommended_ids = []
    
    return recommended_ids

## Generar recomendaciones para un usuario específico

In [ ]:
# recomendación para el usuario id = 126522, utilizando MPNet
user_id = 126522
rec = recommend(book_descripcion_vectors_mpnet, user_id=user_id, topk=15)
rec 

## transacciones pasadas del usuario 

In [ ]:
past_interactions = df_books_interactions.loc[df_books_interactions['user_id'] == user_id]
df_books_small[df_books_small.book_id.isin(past_interactions.book_id.astype(str))][['book_id', 'title', 'description']]

## información de recomendaciones

In [ ]:
df_books_small[df_books_small.book_id.isin(rec)][['book_id', 'title', 'description']]

# Evaluación de las recomendaciones con interacciones de testing 

In [ ]:
# Métricas de evaluación 
# Obtenido de https://gist.github.com/bwhite/3726239

def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])
  
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)

    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

## Evaluación de recomendación con MPNet

In [ ]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

embeddings = book_descripcion_vectors_mpnet
topk = 10 

for i, u in enumerate(df_books_interactions_test.user_id.tolist()):
    
    print(i, end= '\r')
    
    rec = recommend(embeddings, user_id = u, topk=topk)
    rel_vector = [np.isin(df_books_interactions.loc[df_books_interactions['user_id'] == u].book_id.tolist(), [eval(i) for i in rec], assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(df_books_interactions_test)
mean_ndcg /= len(df_books_interactions_test)

time_taken = time.time() - start

In [ ]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

## Evaluación de recomendación con MPNet reducidos con PCA-20

In [ ]:
start = time.time()

mean_map = 0.
mean_ndcg = 0.

embeddings = pca20_mpnet_featvectors
topk = 10 

for i, u in enumerate(df_books_interactions_test.user_id.tolist()):
    
    print(i, end= '\r')
    
    rec = recommend(embeddings, user_id = u, topk=topk)
    rel_vector = [np.isin(df_books_interactions.loc[df_books_interactions['user_id'] == u].book_id.tolist(), [eval(i) for i in rec], assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, topk)

mean_map /= len(df_books_interactions_test)
mean_ndcg /= len(df_books_interactions_test)

time_taken = time.time() - start

In [ ]:
print('MAP ',mean_map)
print('ndcg@10' ,mean_ndcg)
print('tiempo de ejecucion {0:.2f} segs'.format(time_taken))

### Pregunta 5: 
Comente los resultados en términos de tiempo de ejecución y métricas de ranking para los 2 modelos.